In [33]:
import face_recognition
import cv2
import numpy
from PIL import Image, ImageDraw
# This is a demo of running face recognition on a video file and saving the results to a new video file.
#
# PLEASE NOTE: This example requires OpenCV (the `cv2` library) to be installed only to read from your webcam.
# OpenCV is *not* required to use the face_recognition library. It's only required if you want to run this
# specific demo. If you have trouble installing it, try any of the other demos that don't require it instead.

# Open the input movie file
path=r"E:/switch/DL/"
input_movie = cv2.VideoCapture(path+"us.mp4")
length = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT))
print("length: ",length)
if(not input_movie.isOpened()):
    print("open failed!!")
# Create an output movie file (make sure resolution/frame rate matches input video!)
#fourcc = cv2.VideoWriter_fourcc(*'XVID')
fourcc = cv2.VideoWriter_fourcc('M', 'P', '4', '2')
width=int(input_movie.get(3))
hight=int(input_movie.get(4))
output_movie = cv2.VideoWriter('us_output.mp4', fourcc, input_movie.get(5), (width,hight))
# Load some sample pictures and learn how to recognize them.
lmm_image = face_recognition.load_image_file(path+"zbt.jpg")
lmm_face_encoding = face_recognition.face_encodings(lmm_image)[0]
face_locations = face_recognition.face_locations(lmm_image)
print(type(lmm_image),lmm_image.shape,lmm_image.dtype)
print(face_locations)
top,right,bottom,left=face_locations[0]
face_image = lmm_image[top:bottom, left:right]
pil_image = Image.fromarray(face_image)
pil_image.show()
al_image = face_recognition.load_image_file(path+"ds.jpg")
al_face_encoding = face_recognition.face_encodings(al_image)[0]
pil_image2 = Image.fromarray(al_image)
pil_image2.show()
known_faces = [
    lmm_face_encoding,
    al_face_encoding
]

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
frame_number = 0
print("BEGIN")
while True:
    # Grab a single frame of video
    ret, frame = input_movie.read()

    
    frame_number += 1

    # Quit when the input video file ends
    if not ret:
        break

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1].copy()
    
    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        # See if the face is a match for the known face(s)
        match = face_recognition.compare_faces(known_faces, face_encoding, tolerance=0.50)

        # If you had more than 2 faces, you could make this logic a lot prettier
        # but I kept it simple for the demo
        name = None
        if match[0]:
            name = "zhangbt"
        elif match[1]:
            name = "dings"

        face_names.append(name)

    # Label the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        if not name:
            continue

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 25), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)

    # Write the resulting image to the output video file
    if(frame_number%10==0):
        print("Writing frame {} / {}".format(frame_number, length))
    output_movie.write(frame)

# All done!
print("ALL DONE")
input_movie.release()
cv2.destroyAllWindows()

length:  293
<class 'numpy.ndarray'> (1920, 864, 3) uint8
[(665, 788, 1220, 233)]
BEGIN
Writing frame 10 / 293
Writing frame 20 / 293
Writing frame 30 / 293
Writing frame 40 / 293
Writing frame 50 / 293
Writing frame 60 / 293
Writing frame 70 / 293
Writing frame 80 / 293
Writing frame 90 / 293
Writing frame 100 / 293
Writing frame 110 / 293
Writing frame 120 / 293
Writing frame 130 / 293
Writing frame 140 / 293
Writing frame 150 / 293
Writing frame 160 / 293
Writing frame 170 / 293
Writing frame 180 / 293
Writing frame 190 / 293
Writing frame 200 / 293
Writing frame 210 / 293
Writing frame 220 / 293
Writing frame 230 / 293
Writing frame 240 / 293
Writing frame 250 / 293
Writing frame 260 / 293
Writing frame 270 / 293
Writing frame 280 / 293
Writing frame 290 / 293
ALL DONE


In [1]:
#input_movie = cv2.VideoCapture("E:/jupyter-notebook/DLL/output.avi")

from PIL import Image, ImageDraw

import cv2  
import threading
import win32gui,win32con
 
class Producer(threading.Thread):  
    """docstring for ClassName"""  
    def __init__(self,str_rtsp):  
        super(Producer, self).__init__()  
        self.str_rtsp = str_rtsp
        self.play = True
        #通过cv2中的类获取视频流操作对象cap  
        self.cap = cv2.VideoCapture(self.str_rtsp)   
        #调用cv2方法获取cap的视频帧（帧：每秒多少张图片）  
        fps = self.cap.get(cv2.CAP_PROP_FPS)  
        print(fps)  
        #获取cap视频流的每帧大小  
        size = (int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH)),  
                int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))  
        print(size)  
        #定义编码格式mpge-4  
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        #定义视频文件输入对象  
        self.outVideo = cv2.VideoWriter('saveDir.avi',fourcc,fps,size)
        cv2.namedWindow("cap video",0);
    def run(self):  
        print('in producer')   
        while True:  
            ret,image_ = self.cap.read()
            image=image_[:,:,::-1].copy()
            print(ret,image.shape)
            
            if (ret == True):
                if win32gui.FindWindow(None,'cap video'):
                    cv2.imshow('cap video',image)
                    #self.outVideo.write(image)
                else:  
                    self.outVideo.release()
                    self.cap.release()  
                    cv2.destroyAllWindows()
                    break
            break
            cv2.waitKey(1)
            if cv2.waitKey(1) & 0xFF == ord('q'): 
                self.outVideo.release()  
                self.cap.release()  
                cv2.destroyAllWindows()
                break
                # continue
if __name__ == '__main__':  
    print('run program')  
    rtsp_str="E:/jupyter-notebook/DLL/output.avi"
    producer = Producer(rtsp_str)  
    producer.start()

run program
28.999
(544, 960)
in producer
True (960, 544, 3)


In [31]:
cap = cv2.VideoCapture("E:/jupyter-notebook/DLL/us_output.mp4")

while(1):
    ret, frame = cap.read()
    if ret:
        cv2.imshow("capture", frame)
    else:
        break
    if cv2.waitKey(30) &0xFF ==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [2]:
import face_recognition
import cv2
import numpy as np
from collections import deque

In [3]:
import numpy as np
import cv2
#import argparse
from collections import deque


cap=cv2.VideoCapture(0)

pts = deque(maxlen=64)

Lower_green = np.array([110,50,50])
Upper_green = np.array([130,255,255])
while True:
	ret, img=cap.read()
	hsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
	kernel=np.ones((5,5),np.uint8)
	mask=cv2.inRange(hsv,Lower_green,Upper_green)
	mask = cv2.erode(mask, kernel, iterations=2)
	mask=cv2.morphologyEx(mask,cv2.MORPH_OPEN,kernel)
	#mask=cv2.morphologyEx(mask,cv2.MORPH_CLOSE,kernel)
	mask = cv2.dilate(mask, kernel, iterations=1)
	res=cv2.bitwise_and(img,img,mask=mask)
	cnts,heir=cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2:]
	center = None
 
	if len(cnts) > 0:
		c = max(cnts, key=cv2.contourArea)
		((x, y), radius) = cv2.minEnclosingCircle(c)
		M = cv2.moments(c)
		center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
 
		if radius > 5:
			cv2.circle(img, (int(x), int(y)), int(radius),(0, 255, 255), 2)
			cv2.circle(img, center, 5, (0, 0, 255), -1)
		
	pts.appendleft(center)
	for i in range (1,len(pts)):
		if pts[i-1]is None or pts[i] is None:
			continue
		thick = int(np.sqrt(len(pts) / float(i + 1)) * 2.5)
		cv2.line(img, pts[i-1],pts[i],(0,0,225),thick)
		
	
	cv2.imshow("Frame", img)
	cv2.imshow("mask",mask)
	cv2.imshow("res",res)
	
	
	k=cv2.waitKey(30) & 0xFF
	if k==32:
		break
# cleanup the camera and close any open windows
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 